<a href="https://colab.research.google.com/github/dimup/119NER/blob/main/KoBERT_NER_KMOU_for_119NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **구글 드라이브와 연동 및 경로설정**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# KoBERT tokenizer import하기 위한 경로 추가 설정
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/Colab Notebooks']


<br>

# **KoBERT 기반 119NER 모델 구현**
BERT의 다국어 모델인 BERT-multilingual 모델에 한국어를 추가 학습한 SK T-Brain KoBERT를 기반으로한 모델 구현입니다.


<br>
<br>

# **install & import**
tranformers 버전이 최근 갱신되며 기존 코드가 적용되지 않는 오류가 있습니다.
<br>
따라서 3.4.0 버전으로 install을 진행합니다.

In [3]:
!pip install "transformers==3.4.0"

     |████████████████████████████████| 1.3MB 4.1MB/s 
     |████████████████████████████████| 2.9MB 17.9MB/s 
     |████████████████████████████████| 1.2MB 31.9MB/s 
     |████████████████████████████████| 901kB 36.9MB/s 


In [4]:
import tensorflow as tf
import torch
import re
from transformers import BertModel
from transformers import BertTokenizer
import transformers.modeling_bert
from tokenization_kobert import KoBertTokenizer
from transformers import BertForTokenClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import time
import datetime
import random

In [5]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

<br>
<br>

# **데이터 로드**



*   데이터 수집은 한국해양대학교 개체명 코퍼스에서 input, target 각각 약 21000 문장씩 파싱했으며 Training set으로 약 17000문장, Vaildation set으로 약 4000문장을 선정했습니다.
*   119 신고 도메인에 맞추기 위해 '피해 유형'을 나타내는 <EMR> 태그를 부착한 문장 데이터를 추가로 약 1000개 구축했습니다.




### **Input, target 데이터 load**

In [ ]:
# f = open("drive/My Drive/BERT_NER/final_input.txt", 'r')
# f = open("drive/My Drive/BERT_NER/final_input_new_data.txt", 'r')
f = open("drive/My Drive/BERT_NER/for_AI/final_input_new_data_for_AI_201130.txt", 'r')
input_list = f.readlines()
f.close()
print("--예시--")
print(input_list[0:10], "\n 총 문장 수 : ", len(input_list))

--예시--
['여기서 우투수는 우타자를 상대로 0.279, 좌타자를 상대로 0.286의 피안타율을 기록했다.\n', "위쪽 절반은 빨갛게, 나머지 반은 하얗게 칠해진 단단한 공. 미국과 유럽, 일본 등 전 세계 수천만 명이 스마트폰 안에서 던지고 있는 '몬스터 볼'이다.\n", '안 전 대표는 친박(친박근혜)과 친문(친문재인)을 제외한 세력이 합치는 \'제3지대론\'에 대해 "총선 민심이 저희를 깨워 주셨는데 그걸 스스로 부정하는 건 민심에 반하는 것"이라고 일축했다.\n', '얼마 전 안산에 있는 지아네 집을 찾았다.\n', 'tvN 관계자는 1월31일 "술을 마시며 이야기하는 취중진담 콘셉트의 장점을 최대한 살리기 위해 이 같이 결정했다"며 "15세 이상 관람가 때보다 조금 더 편안하고 자유로운 분위기를 전달하기 위해 맞춤 편집한다"고 밝혔다.\n', '슈틸리케 감독은 이날 전주월드컵경기장에서 팬 사인회와 시축으로 팬들을 만나며 K리그 개막을 축하한다.\n', '2015 K리그 클래식은 차두리(서울), 김신욱(울산), 정대세(수원), 이동국 등 내로라하는 스타선수들이 개막 준비를 끝내고 축구팬들을 맞을 채비를 마쳤다.\n', '선불카드 하나로 4명이 무기명으로 그린피 면제 혜택을 받을 수도 있다.\n', "지난 5일 부산에서 열린 문 전 대표의 북콘서트에는 '미생'을 그린 만화가 윤태호, 가수 강산에ㆍ박기영, 방송인 김미화가 참석했다.\n", '반면 2경기 째 승리를 올리지 못한 선두 서울은 6승1무2패, 승점 19로 제자리걸음했다.\n'] 
 총 문장 수 :  18186


In [ ]:
# f = open("drive/My Drive/BERT_NER/final_target.txt", 'r')
# f = open("drive/My Drive/BERT_NER/final_target_new_data.txt", 'r')
f = open("drive/My Drive/BERT_NER/for_AI/final_target_new_data_for_AI_201130.txt", 'r')
target_list = f.readlines()
f.close()
print("--예시--")
print(target_list[0:10], "\n 총 문장 수 : ", len(target_list))

--예시--
['여기서 우투수는 우타자를 상대로 <0.279:PNT>, 좌타자를 상대로 <0.286:PNT>의 피안타율을 기록했다.\n', "위쪽 절반은 빨갛게, 나머지 반은 하얗게 칠해진 단단한 공. <미국:LOC>과 <유럽:LOC>, <일본:LOC> 등 전 세계 <수천만 명:NOH>이 스마트폰 안에서 던지고 있는 '몬스터 볼'이다.\n", '<안:PER> 전 대표는 친박(친<박근혜:PER>)과 친문(친<문재인:PER>)을 제외한 세력이 합치는 \'<제3:NOH>지대론\'에 대해 "총선 민심이 저희를 깨워 주셨는데 그걸 스스로 부정하는 건 민심에 반하는 것"이라고 일축했다.\n', '얼마 전 <안산:LOC>에 있는 <지아:PER>네 집을 찾았다.\n', '<tvN:ORG> 관계자는 <1월31일:DAT> "술을 마시며 이야기하는 취중진담 콘셉트의 장점을 최대한 살리기 위해 이 같이 결정했다"며 "<15세:NOH> 이상 관람가 때보다 조금 더 편안하고 자유로운 분위기를 전달하기 위해 맞춤 편집한다"고 밝혔다.\n', '<슈틸리케:PER> 감독은 이날 <전주월드컵경기장:LOC>에서 팬 사인회와 시축으로 팬들을 만나며 <K리그:POH> 개막을 축하한다.\n', '<2015:DAT> <K리그 클래식:POH>은 <차두리:PER>(<서울:LOC>), <김신욱:PER>(<울산:LOC>), <정대세:PER>(<수원:LOC>), <이동국:PER> 등 내로라하는 스타선수들이 개막 준비를 끝내고 축구팬들을 맞을 채비를 마쳤다.\n', '선불카드 <하나:NOH>로 <4명:NOH>이 무기명으로 그린피 면제 혜택을 받을 수도 있다.\n', "지난 <5일:DAT> <부산:LOC>에서 열린 <문:PER> 전 대표의 북콘서트에는 '<미생:POH>'을 그린 만화가 <윤태호:PER>, 가수 <강산에:PER>ㆍ<박기영:PER>, 방송인 <김미화:PER>가 참석했다.\n", '반면 <2경기:NOH> 째 승리를 올리지 못한 선두 <서울:LOC>은 <6승:NOH><1무:NOH><2패:NOH>, 승점 

## **데이터 전처리 : Input용**
BERT의 Transformer 알고리즘 구조에 맞추기 위해 input 데이터를 전처리하는 과정입니다.

#### **SentencePiece tokenizing, Integer encoding, Padding, Attention masks** 
SentencePiece tokennizing과 정수 인코딩, 패딩 작업을 수행합니다.
<br>
또한, Attention masks를 생성합니다. 이는 학습 시간의 효율을 위해 패딩된 부분을 Attention 구조에 통과시키지 않도록 하는 역할을 합니다.

In [6]:
def preprocessing_input(input_list):

  # CLS와 SEP토큰 부착
  text_CLS = ["[CLS] " + str(txt) + " [SEP]" for txt in input_list]

  # 토크나이저 정의 및 토크나이징
  tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
  tokenized_texts = [tokenizer.tokenize(txt) for txt in text_CLS]
 
  # 토큰 --> 정수 인덱스로 변환
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts] 

  # 입력 토큰의 최대 시퀀스 길이
  MAX_LEN = 128

  # 패딩 수행 : 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움 
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  # 어텐션 마스크 초기화
  attention_masks = []

  # 어텐션 마스크, 값이 있다면 1, 패딩 부분이면 0으로 설정
  # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)
 
  return input_ids, attention_masks, tokenized_texts

In [ ]:
input_ids, attention_masks, tokenized_text = preprocessing_input(input_list)

## **데이터 전처리 : Target용**



#### **SentencePiece tokenizing**
input data와 동일하게 SentencePiece tokenizing을 진행합니다.

In [7]:
def preprocessing_target(input_list, target_list):
  # 토크나이징
  tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
  tokenized_target_text = [tokenizer.tokenize(text_temp) for text_temp in target_list]

  # CLS, SEP가 부착되지 않은 input 데이터
  tokenized_texts_withoutCLS = [tokenizer.tokenize(text) for text in input_list]

  # 밑에 진행되는 작업들까지 수행
  result_target_list = convert_target_to_string(tokenized_target_text)
  prefix_sum_of_token_start_index_list, list_of_ner_tag_list, list_of_ner_text_list, list_of_tuple_ner_start_end_list = searching_boundary(tokenized_texts_withoutCLS, result_target_list)
  list_of_ner_label_list = tagging_label(tokenized_texts_withoutCLS, prefix_sum_of_token_start_index_list, list_of_ner_tag_list, list_of_ner_text_list, list_of_tuple_ner_start_end_list)
  ner_label_list_result = append_CLS_SEP(list_of_ner_label_list)
  return ner_label_list_result

#### **Tokenized input text에 대응되는 개체명 테그셋 만들기**
Input에 대한 각각 토큰과 Target의 개체명 태그를 맞춰주는 작업입니다.

각각의 토큰으로 list에 저장되어 있는 target data를 하나의 문자열로 변환합니다.

In [8]:
def convert_target_to_string(tokenized_target_text):
  result_string_list = []
  # list형식의 tokenized target token들을 하나의 문자열로 변환 
  for target_token in tokenized_target_text:
    result_string = ""
    for tkd in target_token:
      result_string += tkd
    result_string_list.append(result_string)

  return result_string_list

개체명이 나타나는 부분이 어디인지 알아내기 위한 사전준비로 다음을 수행합니다.


1.   하나의 input 문장을 이루는 token들이 각각 몇 번째에 출현하는지 찾음 
2.   이전에 문자열로 변환했던 target 문장에서 정규식을 통해 개체명, 개체명 태그, 개체명이 출현하는 범위를 찾음



In [9]:
def searching_boundary(tokenized_texts_withoutCLS, new_result_target_list):
  prefix_sum_of_token_start_index_list = []
  for tokenized_texts in tokenized_texts_withoutCLS:
    prefix_sum_of_token_start_index = []
    sum = 0
    for i, token in enumerate(tokenized_texts):
        if i == 0:
            prefix_sum_of_token_start_index.append(0)
            sum += len(token)
        else:
            prefix_sum_of_token_start_index.append(sum)
            sum += len(token)

    prefix_sum_of_token_start_index_list.append(prefix_sum_of_token_start_index)


  regex_ner = re.compile('<(.+?):[A-Z]{3}>')

  list_of_ner_tag_list = []
  list_of_ner_text_list = []
  list_of_tuple_ner_start_end_list = []

  for result_string in new_result_target_list:
    filterd_by_regex = regex_ner.finditer(result_string)

    list_of_ner_tag = []
    list_of_ner_text = []
    list_of_tuple_ner_start_end = []

    count_of_match = 0

    for match_item in filterd_by_regex:
        ner_tag = match_item[0][-4:-1]  # <4일간:DUR> -> DUR
        ner_text = match_item[1] # <4일간:DUR> -> 4일간
        start_index = match_item.start() - 6 * count_of_match  # delete previous '<, :, 3 words tag name, >'
        end_index = match_item.end() - 6 - 6 * count_of_match

        list_of_ner_tag.append(ner_tag)
        list_of_ner_text.append(ner_text)
        list_of_tuple_ner_start_end.append((start_index, end_index))
        count_of_match += 1

    list_of_ner_tag_list.append(list_of_ner_tag)
    list_of_ner_text_list.append(list_of_ner_text)
    list_of_tuple_ner_start_end_list.append(list_of_tuple_ner_start_end)

  return prefix_sum_of_token_start_index_list, list_of_ner_tag_list, list_of_ner_text_list, list_of_tuple_ner_start_end_list

구해놓았던 각 input토큰의 시작점과, 실제 개체명의 출현 범위를 비교하며 토큰에 대한 태그를 부착합니다.

In [10]:
def tagging_label(tokenized_texts_withoutCLS, prefix_sum_of_token_start_index_list, list_of_ner_tag_list, list_of_ner_text_list, list_of_tuple_ner_start_end_list):
  list_of_ner_label_list = []
  for i in range(0, len(tokenized_texts_withoutCLS)):
    list_of_ner_label = []
    entity_index = 0
    is_entity_still_B = True
    for tup in zip(tokenized_texts_withoutCLS[i], prefix_sum_of_token_start_index_list[i]): # tokenize된 텍스트와, 각 토큰의 시작점이 tup에 저장됨
        token, index = tup
        if '▁' in token:  # '▁' 이것과 우리가 쓰는 underscore '_'는 서로 다른 문자임
                index += 1  # 토큰이 '▁'를 앞단에 포함한 경우 index 한개 앞으로 당김 # ('▁13', 9) -> ('13', 10)
        
        if entity_index < len(list_of_tuple_ner_start_end_list[i]): # entity_index가 분석 개체명 개수보다 작을 경우
            start, end = list_of_tuple_ner_start_end_list[i][entity_index] # start, end 값은 entity_index 번째의 start, end 값
            if end <= index:  # 토큰을 하나하나 보다가, 인식된 개체명을 넘어가면       # end : 원본 데이터에서 인식된 개체명 엔티티가 끝나는 지점, index : 각 토큰들의 시작점
                is_entity_still_B = True # is_entity_still_B = true로 설정
                entity_index = entity_index + 1 if entity_index + 1 < len(list_of_tuple_ner_start_end_list[i]) else entity_index # 아래 과정을 더하여 다음 인식된 개체명으로 넘어감
                start, end = list_of_tuple_ner_start_end_list[i][entity_index]
            # 인식된 개체명 사이에 index가 도달했을 경우
            if start <= index and index < end: 
                entity_tag = list_of_ner_tag_list[i][entity_index] # 해당 개체명에 대한 태그를 entity_tag에 저장
                if is_entity_still_B is True: # 엔티티의 첫 시작점이라면
                    entity_tag = 'B-' + entity_tag # entity_tag 앞에 B-를 붙임
                    list_of_ner_label.append(entity_tag) # label set에 결과물을 넣음
                    is_entity_still_B = False # is_entity_still_B = False로 설정, 이제부터는 시작점이 아니므로
                else: # 시작점이 아닐 경우
                    entity_tag = 'I-' + entity_tag  # entity_tag 앞에 I-를 붙임
                    list_of_ner_label.append(entity_tag) # label set에 넣음
            else: # 인식된 개체명이 없을 경우
                is_entity_still_B = True # is_entity_still_B = True로 설정
                entity_tag = 'O' # entity_tag = O로 변경
                list_of_ner_label.append(entity_tag) # label set에 넣음

        else:
            entity_tag = 'O'
            list_of_ner_label.append(entity_tag)
    list_of_ner_label_list.append(list_of_ner_label)
  return list_of_ner_label_list

결과물에 [SEP]와 [CLS] 태그를 부착합니다.
<br>

앞서 Input data에도 부착된 이 태그들은 BERT 모델 형식에 맞추기 위함으로, [CLS]는 문장의 시작을 나타내고 [SEP]는 문장이 분리되는 지점(문장의 끝)을 나타냅니다.

In [11]:
def append_CLS_SEP(list_of_ner_label_list):
  for ner_label in list_of_ner_label_list:
    ner_label.append("[SEP]")
    ner_label.insert(0, "[CLS]")
  return list_of_ner_label_list

In [ ]:
ner_label_list_result = preprocessing_target(input_list, target_list)

### Target labels에 대한 Integer encoding & padding작업

target label(tagset) 에 대한 정수 인코딩, 패딩 작업입니다. 
<br>
Tokenized input data에서 사용한 정수 인코딩 방법이 제대로 적용되지 않는 오류가 있어 다음과 같이 tagset에 대한 {태그:숫자값} dictionary를 임의로 생성하였습니다. 

In [12]:
# '피해 유형'을 나타내는 EMR 태그까지 생성
tag_dict = {'[PAD]' : 0, '[CLS]': 1, '[SEP]': 2, 'O' : 3,
            'B-PER': 4, 'I-PER': 5, 
            'B-ORG': 6, 'I-ORG': 7,
            'B-LOC': 8, 'I-LOC': 9, 
            'B-POH': 10, 'I-POH': 11,
            'B-DAT': 12, 'I-DAT': 13,
            'B-TIM': 14, 'I-TIM': 15,
            'B-DUR': 16, 'I-DUR': 17,
            'B-MNY': 18, 'I-MNY': 19,
            'B-PNT': 20, 'I-PNT': 21, 
            'B-NOH': 22, 'I-NOH': 23,
            'B-EMR': 24, 'I-EMR': 25}

{숫자값:태그}을 갖는 역순의 dict입니다. 이후 결과 출력을 위해 사용됩니다.

In [13]:
tag_dict_decode = inv_map = {v: k for k, v in tag_dict.items()}

생성한 dict를 통해 label(tagset)에 대한 정수 인코딩을 진행합니다.

In [ ]:
labels_ids = []

for label_list in ner_label_list_result:
  labels_ids_encoding = [tag_dict.get(x) for x in label_list]  
  labels_ids.append(labels_ids_encoding)

Input data와 같은 길이로 padding을 진행합니다.

In [ ]:
MAX_LEN = 128

labels_ids = pad_sequences(labels_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# **Modeling**

#### **데이터 구조 변환 및 Training 준비**

준비한 각 데이터를 Pytorch의 Tensor라는 자료구조 형식으로 변환합니다. Hugging face의 Pytorch 기반 라이브러리를 사용하기 때문입니다.

In [ ]:
model_inputs = torch.tensor(input_ids)
model_labels = torch.tensor(labels_ids)
model_masks = torch.tensor(attention_masks)

In [ ]:
# 배치 사이즈 (학습 한 번에 들어갈 데이터의 사이즈)
batch_size = 8

# Pytorch의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터를 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
model_data = TensorDataset(model_inputs, model_masks, model_labels)
model_sampler = RandomSampler(model_data)
model_dataloader = DataLoader(model_data, sampler=model_sampler, batch_size=batch_size)

Model을 생성하기 전, GPU연산을 해야 하기 때문에 GPU에 대한 정보를 파악하고, GPU 디바이스 설정을 진행합니다.

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
    
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


Model을 생성합니다.

In [ ]:
# Token 분류를 위한 KoBERT 모델 생성
model = BertForTokenClassification.from_pretrained('monologg/kobert', num_labels = 26, 
                                                   output_attentions = False, output_hidden_states = False)
model.cuda()

# **Training**

#### **Optimizer & Hyper parameters 설정**

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 50

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(model_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성 (overfitting 방지)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

#### **Training 진행**

In [ ]:
# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(model_dataloader):

        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        # 로스 구함
        loss = outputs[0]
        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()
        # 스케줄러로 학습률 감소
        scheduler.step()
        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(model_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")



training 완료한 모델을 저장합니다.

In [ ]:
torch.save(model, 'drive/My Drive/BERT_NER/KoBERTmodel_for_AI_201130.pt')

# **Evaluation**

#### **평가를 위한 Input, Target 데이터 load**

In [14]:
f = open("drive/My Drive/BERT_NER/test_input_final.txt", 'r')
test_input_list = f.readlines()
f.close()
print("--예시--")
print(test_input_list[0:10], "\n 총 문장 수 : ", len(test_input_list))

--예시--
['현대건설은 예선A조 첫 경기서 GS칼텍스를 3-2(25-23 24-26 22-25 25-20 15-13)로 제압했다 ．\n', '대한항공 "한전 꿇어"\n', '한전은 경기가 시작되자마자 강력한 서브로 대한항공 수비진을 흔든 뒤 이병주, 최귀동의 공격을 앞세워 첫 세트를 따내며 불의의 일격을 가했다 .\n', '체육진흥투표권 수탁사업자인 스포츠토토㈜(www.sportstoto.co.kr)는 28일 개막한 2008 KOVO컵 IBK기업은행배 양산프로배구대회를 대상으로 새로운 방식으로 변경된 배구토토 매치게임을 본격 발매한다고 밝혔다 .\n', '●하이원컵 SBS 채리티여자오픈(강원도 정선 하이원골프장)\n', '1공태욱(김해건설공)\n', '니시코리(일본) 기권승 카라누시치(크로아티아), 루옌순(대만) 1(1-6 6-4 6-7 4-6)3 치폴라(이스테이트)\n', '공 감독은 "달리 어쩌겠나 .\n', '한전은 등록선수 9명으로 2008 양산프로배구대회(양산실내체육관)에 나왔다 .\n', '경기내용은 좋아지겠지만 문성민이 와도 금세 팀이 달라지는 것이 아니다 .\n'] 
 총 문장 수 :  4700


In [15]:
f = open("drive/My Drive/BERT_NER/test_target_final.txt", 'r')
test_target_list = f.readlines()
f.close()
print("--예시--")
print(test_target_list[0:10], "\n 총 문장 수 : ", len(test_target_list))

--예시--
['<현대건설:ORG>은 예선<A조:ORG> 첫 경기서 <GS칼텍스:ORG>를 <3-2:NOH>(<25-23:NOH> <24-26:NOH> <22-25:NOH> <25-20:NOH> <15-13:NOH>)로 제압했다 ．\n', '<대한항공:ORG> "<한전:ORG> 꿇어"\n', '<한전:ORG>은 경기가 시작되자마자 강력한 서브로 <대한항공:ORG> 수비진을 흔든 뒤 <이병주:PER>, <최귀동:PER>의 공격을 앞세워 첫 세트를 따내며 불의의 일격을 가했다 .\n', '체육진흥투표권 수탁사업자인 <스포츠토토㈜:POH>(<www.sportstoto.co.kr:POH>)는 <28일:DAT> 개막한 <2008:DAT> <KOVO컵 IBK기업은행배 양산프로배구대회:POH>를 대상으로 새로운 방식으로 변경된 배구토토 매치게임을 본격 발매한다고 밝혔다 .\n', '●<하이원컵 SBS 채리티여자오픈:POH>(<강원도:LOC> <정선 하이원골프장:LOC>)\n', '1<공태욱:PER>(<김해건설공:ORG>)\n', '<니시코리:PER>(<일본:LOC>) 기권승 <카라누시치:PER>(<크로아티아:LOC>), <루옌순:PER>(<대만:LOC>) 1(<1-6:NOH> <6-4:NOH> <6-7:NOH> <4-6:NOH>)<3:NOH> <치폴라:PER>(<이스테이트:LOC>)\n', '<공:PER> 감독은 "달리 어쩌겠나 .\n', '<한전:ORG>은 등록선수 <9명:NOH>으로 <2008:DAT> <양산프로배구대회:POH>(<양산실내체육관:LOC>)에 나왔다 .\n', '경기내용은 좋아지겠지만 <문성민:PER>이 와도 금세 팀이 달라지는 것이 아니다 .\n'] 
 총 문장 수 :  4700


#### 평가를 위한 데이터 전처리 : Input용
기존 학습 데이터의 전처리 과정과 동일합니다.

In [16]:
test_input_ids, test_masks_ids, tokenized_texts = preprocessing_input(test_input_list)

#### 평가를 위한 데이터 전처리 : Target용

In [17]:
test_target = preprocessing_target(test_input_list, test_target_list)

#### Evaluation 진행

저장한 Model을 불러옵니다.

In [18]:
model = torch.load('drive/My Drive/BERT_NER/KoBERTmodel_for_AI_201130.pt')

GPU 디바이스 설정을 학습때와 동일하게 진행합니다.

In [19]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
    
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


forward를 수행합니다.

In [20]:
# 평가모드로 변경
model.eval()
# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0



tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
pred_labels = [] # 예측 태그셋을 담을 리스트
# 데이터로더에서 배치만큼 반복하여 가져옴
for i in range(0, len(test_input_list)):
    stopFlag = False
    test_inputs = torch.tensor([test_input_ids[i]])
    test_masks = torch.tensor([test_masks_ids[i]])

    # 배치를 GPU에 넣음
    b_input_ids = test_inputs.to(device)
    b_input_mask = test_masks.to(device)
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = np.argmax(outputs[0].to('cpu').numpy(), axis=2)
    # join bpe split tokens
    tokens = tokenizer.convert_ids_to_tokens(b_input_ids.to('cpu').numpy()[0])
    new_labels = []
    for token, label_idx in zip(tokens, logits[0]):
        if stopFlag == True:
          break
        if token == '[SEP]': # 패딩 전까지의 출력만을 보기 위해
          stopFlag = True
        new_labels.append(tag_dict_decode[label_idx])

    pred_labels.append(new_labels)

토큰간 정확도(Accuracy) 측정을 진행합니다.

In [21]:
num, total = 0, 0
for i in range(0, len(pred_labels)):
  # 고정으로 들어가 있는 [CLS], [SEP] 는 제외함
  for pred, target in zip(pred_labels[i][1:-1], test_target[i][1:-1]):
    if pred == target:
      num +=1
  total += len(pred_labels[i])-2 # [CLS], [SEP] 태그 수 빼기

print("Accuracy : ", num/total*100,"%")

Accuracy :  92.38126530856084 %


**하지만, 과연 토큰 간 Accuracy 측정이 모델의 정확도를 평가하는데에 적절하다고 볼 수 있을까요?**

- 예를들어 "경복궁역"에서 "경복" 까지만 <위치>로 예측했다면, 50%는 맞췄다고 보는 것이 과연 올바를까요?
- 단언하기엔 애매하지만, 현업에서 사용하기엔 적절한 평가 방법은 아닌 것 같습니다.

따라서 아래와 같이 토큰 간 비교가 아닌, 각각의 개체명 전체에 대한 정확도 평가 방법을 수행해보았습니다.

In [22]:
num, total = 0, 0
for i in range(0, len(pred_labels)):
  start_index = -1
  end_index = -1
  for t in range(1, len(pred_labels[i])-1):

    if pred_labels[i][t][0] == "B" and start_index == -1:
      start_index = t
    elif start_index != -1 and pred_labels[i][t][0] == "O":
      end_index = t
      if test_target[i][start_index:end_index] == pred_labels[i][start_index:end_index]:
        num += 1
      total += 1
      start_index = -1

    elif start_index != -1 and pred_labels[i][t][0] == "B":
      end_index = t
      if test_target[i][start_index:end_index] == pred_labels[i][start_index:end_index]:
        num += 1
      total += 1
      start_index = t

print("Accuracy : ", num / total * 100, "%")

Accuracy :  85.46427836112827 %


# **Testing**
임의의 문장이 모델을 통과한 결과를 가공하여 출력 형태를 맞춘 후, 올바르게 분석되는지 확인합니다. 

#### **입력 문장에 대한 전처리**
학습 데이터 중 Input data에 대한 전처리 과정과 동일합니다.

In [ ]:
# 입력 데이터 전처리
def convert_input_data(sentences):
    text_CLS = ["[CLS] " + str(txt) + " [SEP]" for txt in sentences]
    tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
    # 토크나이징
    tokenized_texts = [tokenizer.tokenize(sent) for sent in text_CLS]
    MAX_LEN = 128 # MAX_LEN 설정
    # 정수 인코딩 및 패딩 진행
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # 어텐션 마스크 설정
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

#### **모델 통과 및 결과 가공 수행**

In [ ]:
# 문장 테스트
def test_sentences(sentences):
    stopFlag = False
    tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
    # 평가모드로 변경
    model.eval()
    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)
    if torch.cuda.is_available():    
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask
                        )
    logits = np.argmax(outputs[0].to('cpu').numpy(), axis=2)
    
    # join bpe split tokens
    tokens = tokenizer.convert_ids_to_tokens(b_input_ids.to('cpu').numpy()[0])
    new_tokens, new_labels = [], []
    for token, label_idx in zip(tokens, logits[0]):
        if stopFlag == True:
          break
        if token == '[SEP]': # 패딩 전까지의 출력만을 보기 위해
          stopFlag = True
        new_labels.append(tag_dict_decode[label_idx])
        new_tokens.append(token)

    # return logits
    return new_labels, new_tokens

#### **결과 확인**

In [ ]:
input_text = '상명대학교의 송재우는 두레미담에서 11시 30분에 점심을 먹었다.'
new_label, new_token = test_sentences([input_text])

for token, label in zip(new_token, new_label): 
    print("{}\t{}".format(label, token))

[CLS]	[CLS]
B-ORG	▁상
I-ORG	명
I-ORG	대학교
O	의
B-PER	▁송
I-PER	재
I-PER	우는
B-POH	▁두
I-POH	레
I-POH	미
I-POH	담
O	에서
B-TIM	▁11
I-TIM	시
I-TIM	▁30
I-TIM	분
O	에
O	▁점
O	심을
O	▁먹
O	었다
[SEP]	.
[SEP]	[SEP]


In [ ]:
input_text = '여기가 지금 스타벅스 광화문점인데 불이 갑자기 났거든요? 얼른 와주세요'
new_label, new_token = test_sentences([input_text])

for token, label in zip(new_token, new_label): 
    print("{}\t{}".format(label, token))

[CLS]	[CLS]
O	▁여기
O	가
O	▁지금
B-ORG	▁스타
I-ORG	벅
I-ORG	스
B-LOC	▁광
I-LOC	화
I-LOC	문
I-LOC	점
O	인데
B-EMR	▁불
I-EMR	이
I-EMR	▁갑자기
I-EMR	▁
I-EMR	났
I-EMR	거
I-EMR	든
I-EMR	요
O	?
O	▁
O	얼
O	른
O	▁
O	와
O	주세요
[SEP]	[SEP]
